In [1]:
# Import required libraries
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\ramakant.raju\AppData\Local\Temp\ipykernel_10628\1367057610.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ramakant.raju\AppData\Local\Temp\ipykernel_10628\1367057610.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [37]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard 115',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                # Function decorator to specify function input and output
                                html.Div([
                                    dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': x, 'value': x} for x in spacex_df["Launch Site"].unique()
                                                ]+
                                                 [{'label': 'All Sites', 'value': 'ALL'}],
                                                value='ALL',
                                                placeholder="Select a Launch Site",
                                                searchable=True
                                                )
                                        ]),
                                html.Br(),
                                html.H1(children = 'Pie chart app',
                                        style = {'textAlign': 'centre',
                                                 'color': '#503D36',
                                                'font-size': '14',
                                                }
                                        ),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div([
                                    dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                value=[100, 5000])
                                    ]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df["class"]==1]
        filtered_df = filtered_df.groupby("Launch Site").count().reset_index()
        fig = px.pie(filtered_df, 
                     values='Flight Number', 
                     names='Launch Site', 
                     title='All Launch sites')
        return fig
    else:
# return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df["Launch Site"]==entered_site]
        filtered_df = filtered_df.groupby("class").count().reset_index()
        fig = px.pie(filtered_df, 
                     values='Launch Site', 
                     names='class', 
                     title="Success rate of " + entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id="payload-slider", component_property="value"),
              Input(component_id='site-dropdown', component_property='value')
             )

def get_scatter_plot(payload, site):
    if site == "ALL":
        df1 = spacex_df
        fig = px.scatter(df1, 
                            x="Payload Mass (kg)", 
                            y="class", 
                            color = "Booster Version Category",
                            title="all Sites")
        fig.update_layout(xaxis_range = payload)
        return fig
    else:
        df1 = spacex_df[spacex_df["Launch Site"]==site]
        fig = px.scatter(df1, 
                            x="Payload Mass (kg)", 
                            y="class", 
                            color = "Booster Version Category",
                            title="Scatter plot for " + site)
        fig.update_layout(xaxis_range = payload)
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server(mode = "external")

Dash app running on http://127.0.0.1:8050/
